# FastF1 Notes
## Track Status Codes
1. Track Clear
1. Yellow Flag
1. Unknown
1. Safety Car
1. Red Flag
1. VSC Deployed
1. VSC Ending

## Misc
* SignalRClient should be started 2-3 minutes before session start to collect all live data
* VSC laps are 130% time length of hotlaps
* isAccurate Tracks:
    * Lap start, end time are synced with other laps
    * Not an inlap or outlap
    * Is not first car after safety car ends
    * Has lap time, sum of sector times matches lap time
## Overall todo
* Figure out how to interpret in/out of pit times

In [1]:
import fastf1 as f1
import fastf1.livetiming.client as LiveClient
import fastf1.livetiming.data as LiveData
import pandas

# Race data test

In [2]:
# Race data test
session = f1.get_session(2025, 1, "R") # optional: backend = "fastf1"
session.load(telemetry = False, weather = False)

req         WARNING 	DEFAULT CACHE ENABLED! (223.49 MB) /home/studio-lab-user/.cache/fastf1
core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']


In [19]:
session.laps.columns  # SpeedI are top speeds (sector 1, sector 2, finish line, longest straight)

Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime',
       'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
       'FastF1Generated', 'IsAccurate'],
      dtype='object')

In [3]:
# Race data test part 2
x = session.laps[["Driver", "LapNumber", "Stint", "LapStartTime", "Time", "LapTime", "PitOutTime", "PitInTime", "Compound", "TrackStatus", 
                  "IsAccurate", "FastF1Generated"]]

# Convert times to seconds
COLUMNS_TIME_CONVERT = ["LapStartTime", "Time", "LapTime", "PitOutTime", "PitInTime"]
for var in COLUMNS_TIME_CONVERT:
    #x.loc[var] = x[var].apply(lambda x:"" if x == "" else x.total_seconds())
    x[var] = x[var].dt.total_seconds()

#x.to_csv(f"{str(session.event.EventDate)[0:4]}-{session.event.Location.replace(" ", "")}-{session.name}.csv")
x.head()

/tmp/ipykernel_96/740910187.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[var] = x[var].dt.total_seconds()


,Driver,LapNumber,Stint,LapStartTime,Time,LapTime,PitOutTime,PitInTime,Compound,TrackStatus,IsAccurate,FastF1Generated
0,VER,1.0,1.0,4260.355,4380.002,119.392,NaN,NaN,INTERMEDIATE,124,False,False
1,VER,2.0,1.0,4380.002,4549.358,NaN,NaN,4538.205,INTERMEDIATE,4,False,False
2,VER,3.0,2.0,4549.358,4711.526,NaN,4551.658,4700.223,INTERMEDIATE,4,False,False
3,VER,4.0,3.0,4711.526,4867.226,NaN,4714.029,4856.543,INTERMEDIATE,4,False,False
4,VER,5.0,4.0,4867.226,5010.835,143.609,4869.534,NaN,INTERMEDIATE,4,False,False


In [10]:
type(x.loc[0, "Time"])

pandas._libs.tslibs.timedeltas.Timedelta

# Quali Data Test

In [5]:
# Quali data test
session_q = f1.get_session(2025, 7, "Q")
session_q.load(telemetry = False, weather = False)

core           INFO 	Loading data for Emilia Romagna Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['81', '1', '63', '4', '14', '55', '23', '18', '6', '10', '16', '44', '12', '5', '43', '30', '27', '31', '87', '22']


In [6]:
# Quali data test part 2
session_q.laps[["Driver", "LapNumber", "Stint", "LapStartTime", "Time", "LapTime", "PitOutTime", "PitInTime", "Compound", "TrackStatus", 
                "IsAccurate", "FastF1Generated"]]

,Driver,LapNumber,Stint,LapStartTime,Time,LapTime,PitOutTime,PitInTime,Compound,TrackStatus,IsAccurate,FastF1Generated
0,PIA,1.0,1.0,0 days 00:18:52.826000,0 days 00:37:17.211000,NaT,0 days 00:18:52.826000,0 days 00:20:59.299000,SOFT,125,False,False
1,PIA,2.0,2.0,0 days 00:37:17.211000,0 days 00:38:52.325000,NaT,0 days 00:37:17.211000,NaT,SOFT,1,False,False
2,PIA,3.0,2.0,0 days 00:38:52.325000,0 days 00:40:07.825000,0 days 00:01:15.500000,NaT,NaT,SOFT,1,True,False
3,PIA,4.0,2.0,0 days 00:40:07.825000,0 days 00:41:51.898000,0 days 00:01:44.073000,NaT,NaT,SOFT,1,True,False
4,PIA,5.0,2.0,0 days 00:41:51.898000,0 days 00:43:51.844000,0 days 00:01:59.946000,NaT,NaT,SOFT,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
276,BEA,6.0,2.0,0 days 00:38:10.947000,0 days 00:39:47.412000,0 days 00:01:36.465000,NaT,0 days 00:39:42.993000,SOFT,1,False,False
277,BEA,7.0,3.0,0 days 00:39:47.412000,0 days 00:44:46.299000,NaT,0 days 00:43:09.417000,NaT,SOFT,1,False,False
278,BEA,8.0,3.0,0 days 00:44:46.299000,0 days 00:46:02.376000,0 days 00:01:16.077000,NaT,NaT,SOFT,12,True,False
279,TSU,1.0,1.0,0 days 00:17:15.622000,0 days 00:18:54.857000,NaT,0 days 00:17:15.622000,NaT,SOFT,1,False,False


In [8]:
session_q.laps.loc[session_q.laps['Driver'] == "PIA"].head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 00:37:17.211000,PIA,81,NaT,1.0,1.0,0 days 00:18:52.826000,0 days 00:20:59.299000,NaT,NaT,...,True,McLaren,0 days 00:18:52.826000,NaT,125,NaN,False,,False,False
1,0 days 00:38:52.325000,PIA,81,NaT,2.0,2.0,0 days 00:37:17.211000,NaT,NaT,0 days 00:00:37.549000,...,False,McLaren,0 days 00:37:17.211000,NaT,1,NaN,False,,False,False
2,0 days 00:40:07.825000,PIA,81,0 days 00:01:15.500000,3.0,2.0,NaT,NaT,0 days 00:00:23.868000,0 days 00:00:26.238000,...,False,McLaren,0 days 00:38:52.325000,NaT,1,NaN,False,,False,True
3,0 days 00:41:51.898000,PIA,81,0 days 00:01:44.073000,4.0,2.0,NaT,NaT,0 days 00:00:36.783000,0 days 00:00:34.663000,...,False,McLaren,0 days 00:40:07.825000,NaT,1,NaN,False,,False,True
4,0 days 00:43:51.844000,PIA,81,0 days 00:01:59.946000,5.0,2.0,NaT,NaT,0 days 00:00:48.295000,0 days 00:00:35.407000,...,False,McLaren,0 days 00:41:51.898000,NaT,1,NaN,False,,False,True
5,0 days 00:45:30.145000,PIA,81,0 days 00:01:38.301000,6.0,2.0,NaT,0 days 00:45:17.949000,0 days 00:00:23.971000,0 days 00:00:26.074000,...,False,McLaren,0 days 00:43:51.844000,NaT,1,NaN,False,,False,False
6,0 days 01:16:04.317000,PIA,81,NaT,7.0,3.0,0 days 01:14:18.477000,NaT,NaT,0 days 00:00:38.590000,...,True,McLaren,0 days 00:45:30.145000,NaT,125,NaN,False,,False,False
7,0 days 01:17:19.558000,PIA,81,0 days 00:01:15.241000,8.0,3.0,NaT,NaT,0 days 00:00:23.911000,0 days 00:00:25.881000,...,True,McLaren,0 days 01:16:04.317000,NaT,1,NaN,False,,False,True
8,0 days 01:19:05.105000,PIA,81,0 days 00:01:45.547000,9.0,3.0,NaT,0 days 01:19:00.654000,0 days 00:00:33.343000,0 days 00:00:33.213000,...,True,McLaren,0 days 01:17:19.558000,NaT,1,NaN,False,,False,False
9,0 days 01:26:27.131000,PIA,81,NaT,10.0,4.0,0 days 01:24:46.527000,NaT,NaT,0 days 00:00:33.315000,...,False,McLaren,0 days 01:19:05.105000,NaT,1,NaN,False,,False,False


# Data stream test

In [ ]:
# Live data stream test
stream = LiveClient.SignalRClient("livetest.txt", filemode='w', debug=False, timeout=300, logger=None)
await stream.async_start() # Client gets disconnected after 2 hours, need to run again to start new instance in event of false start/red flag

In [1]:
# Live data stream reading test
live_data = LiveData.LiveTimingData("livetest.txt")

NameError: name 'LiveData' is not defined

In [ ]:
live_data.list_categories()

In [ ]:
live_data.get("Position.z")

# Weather/Telemetry Test

In [3]:
# Weather/telemetry test
session = f1.get_session(2025, 1, "R") # optional: backend = "fastf1"
session.load(telemetry = True, weather = True)

req         WARNING 	DEFAULT CACHE ENABLED! (132.41 MB) /home/studio-lab-user/.cache/fastf1
core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished lo

In [3]:
session.weather_data  # New weather data every minute

,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,0 days 00:00:07.776000,15.8,91.0,1009.2,True,18.8,211,2.6
1,0 days 00:01:07.777000,15.7,92.0,1009.2,True,18.8,273,2.6
2,0 days 00:02:07.763000,15.7,92.0,1009.0,True,18.8,271,2.2
3,0 days 00:03:07.777000,15.7,90.0,1009.1,True,18.8,261,5.3
4,0 days 00:04:07.778000,15.7,90.0,1009.4,True,18.8,246,5.6
...,...,...,...,...,...,...,...,...
173,0 days 02:53:09.006000,16.1,72.0,1010.4,False,18.7,170,1.4
174,0 days 02:54:09.025000,16.2,72.0,1010.6,False,18.7,216,2.4
175,0 days 02:55:09.010000,16.2,71.0,1010.4,False,18.7,215,2.8
176,0 days 02:56:09.020000,16.2,71.0,1010.4,False,18.7,215,2.8


In [8]:
session.car_data

{'4':                          Date  RPM  Speed  nGear  Throttle  Brake  DRS Source  \
 0     2025-03-16 03:12:38.929  0.0    0.0      0       0.0  False    0    car   
 1     2025-03-16 03:12:39.204  0.0    0.0      0       0.0  False    0    car   
 2     2025-03-16 03:12:39.444  0.0    0.0      0       0.0  False    0    car   
 3     2025-03-16 03:12:39.724  0.0    0.0      0       0.0  False    0    car   
 4     2025-03-16 03:12:39.924  0.0    0.0      0       0.0  False    0    car   
 ...                       ...  ...    ...    ...       ...    ...  ...    ...   
 38416 2025-03-16 06:04:33.849  0.0    0.0      0     104.0   True    0    car   
 38417 2025-03-16 06:04:34.369  0.0    0.0      0     104.0   True    0    car   
 38418 2025-03-16 06:04:34.528  0.0    0.0      0     104.0   True    0    car   
 38419 2025-03-16 06:04:34.768  0.0    0.0      0     104.0   True    0    car   
 38420 2025-03-16 06:04:35.009  0.0    0.0      0     104.0   True    0    car   
 
         

In [9]:
session.pos_data

{'4':                          Date   Status       X       Y     Z Source  \
 0     2025-03-16 03:11:55.448  OnTrack     0.0     0.0   0.0    pos   
 1     2025-03-16 03:11:55.808  OnTrack     0.0     0.0   0.0    pos   
 2     2025-03-16 03:11:56.068  OnTrack     0.0     0.0   0.0    pos   
 3     2025-03-16 03:11:56.268  OnTrack     0.0     0.0   0.0    pos   
 4     2025-03-16 03:11:56.508  OnTrack     0.0     0.0   0.0    pos   
 ...                       ...      ...     ...     ...   ...    ...   
 39554 2025-03-16 06:04:34.048  OnTrack -1012.0 -1309.0  82.0    pos   
 39555 2025-03-16 06:04:34.368  OnTrack -1012.0 -1309.0  82.0    pos   
 39556 2025-03-16 06:04:34.788  OnTrack -1012.0 -1309.0  82.0    pos   
 39557 2025-03-16 06:04:35.008  OnTrack -1012.0 -1309.0  82.0    pos   
 39558 2025-03-16 06:04:35.228  OnTrack -1012.0 -1309.0  82.0    pos   
 
                         Time            SessionTime  
 0     0 days 00:04:32.829000 0 days 00:04:32.829000  
 1     0 days 00:04

In [4]:
# Session telemetry, only supports one driver at a time
tel_test = session.laps[session.laps["Driver"] == "VER"].telemetry
tel_test[tel_test.Speed > 0].head()

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
8,2025-03-16 04:18:23.748,0 days 01:11:01.129000,10,0.086667,0 days 00:00:00.774000,7120.000000,4.533333,2,16.0,False,1,pos,0.245100,8.213403e-07,OnTrack,-943.000000,-1573.000000,86.000000
9,2025-03-16 04:18:23.852,0 days 01:11:01.233000,16,0.000000,0 days 00:00:00.878000,6392.000000,8.000000,2,16.0,False,1,car,0.533333,1.787220e-06,OnTrack,-944.551852,-1571.571139,85.999995
10,2025-03-16 04:18:24.052,0 days 01:11:01.433000,16,0.000000,0 days 00:00:01.078000,4992.000000,13.000000,2,16.0,False,1,car,1.255556,4.207415e-06,OnTrack,-948.518464,-1569.040453,85.999984
11,2025-03-16 04:18:24.148,0 days 01:11:01.529000,16,0.026667,0 days 00:00:01.174000,4915.885714,15.057143,2,16.0,False,1,pos,1.703031,5.706921e-06,OnTrack,-951.000000,-1567.000000,86.000000
12,2025-03-16 04:18:24.308,0 days 01:11:01.689000,16,0.071111,0 days 00:00:01.334000,4789.028571,18.485714,2,16.0,False,1,pos,2.588290,8.673460e-06,OnTrack,-956.000000,-1562.000000,86.000000


In [5]:
session.laps.iloc[0].get_weather_data()  # Type: series

Time             0 days 01:11:08.207000
AirTemp                            15.7
Humidity                           74.0
Pressure                         1010.2
Rainfall                          False
TrackTemp                          19.2
WindDirection                       253
WindSpeed                           3.9
Name: 71, dtype: object

In [6]:
session.laps.iloc[0]

Time                      0 days 01:13:00.002000
Driver                                       VER
DriverNumber                                   1
LapTime                   0 days 00:01:59.392000
LapNumber                                    1.0
Stint                                        1.0
PitOutTime                                   NaT
PitInTime                                    NaT
Sector1Time                                  NaT
Sector2Time               0 days 00:00:20.705000
Sector3Time               0 days 00:00:55.250000
Sector1SessionTime                           NaT
Sector2SessionTime        0 days 01:12:04.853000
Sector3SessionTime        0 days 01:13:00.058000
SpeedI1                                    249.0
SpeedI2                                    292.0
SpeedFL                                    247.0
SpeedST                                    215.0
IsPersonalBest                             False
Compound                            INTERMEDIATE
TyreLife            